In [1]:
'''
Loading libraries
'''
import numpy as np
import pandas as pd
import os
import datetime
import warnings
import itertools

import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline

import sys
warnings.filterwarnings("ignore")

#  ARIMA Model

In [5]:
train_data_path =  'yds_train2018.csv'
train_df = pd.read_csv(train_data_path) #loading train data

test_data_path =  'yds_test2018.csv'
test_df = pd.read_csv(test_data_path) # loading test data

temp = test_df.groupby('Country').agg('size').reset_index() #
temp.columns = ['Country', 'No_of_Months']
forecast_period = temp['No_of_Months'].max()*5

In [6]:
'''
As the train data in on Merchant level, I doppped 'Merchant_ID' and rolled up the data to Product level by summing up the Sales values.
'''
train_df.drop('Merchant_ID', axis=1, inplace=True)
train_df = train_df.groupby(['Country', 'Year', 'Month','Week', 'Product_ID'])
train_df = train_df.agg('sum')['Sales'].reset_index()
train_df.columns = ['Country', 'Year', 'Month','Week', 'Product_ID', 'Monthly_Productwise_sales']

In [7]:
'''
To include time dimension in the train dataset, I added 'Day' (day of the Year) feature. Then formatted the string to datetime.
'''
train_df['Day'] = ((train_df['Week'] - 1)*7)+1
train_df['Date'] = train_df['Year'].astype(str) + '/' + train_df['Month'].astype(str) + '/' + train_df['Day'].astype(str)
train_df['Date'] = pd.to_datetime(train_df['Date'], format = '%Y/%m/%j')

In [8]:
'''
The information of Year, Month and Week are captured in  Date feature. Hence I droppped it. Added Date feature as the Index of dataframe.
'''
train_df.drop('Year', axis=1, inplace=True)
train_df.drop('Month', axis=1, inplace=True)
train_df.drop('Week', axis=1, inplace=True)
train_df.drop('Day', axis=1, inplace=True)

train_df.index = train_df['Date']
train_df.drop('Date', axis=1, inplace=True)
train_df.columns = ['Country', 'Product_ID', 'Sales']
train_df.head()

,Country,Product_ID,Sales
Date,,,
2013-01-01,Argentina,1,4335975.00
2013-01-01,Argentina,2,395846.92
2013-01-08,Argentina,1,9753975.00
2013-01-08,Argentina,2,1449848.96
2013-01-15,Argentina,1,10309950.00


In [9]:
'''
To bring the data down to datetime, the dataset was groupped with Country, Product_ID and stored each group in 'dataframes' list. 
ARIMA can be applied at this level (to each group seperately).
'''
temp = train_df.groupby(['Country','Product_ID'])
df_names = []
dataframes = []
for name, group in temp:
    df_names.append('train_'+name[0]+'_'+str(name[1]))
    dataframes.append(group)

In [11]:
dataframes[0].head()

,Country,Product_ID,Sales
Date,,,
2013-01-01,Argentina,1,4335975.0
2013-01-08,Argentina,1,9753975.0
2013-01-15,Argentina,1,10309950.0
2013-01-22,Argentina,1,7484400.0
2013-01-29,Argentina,1,2461725.0


In [12]:
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 7) for x in list(itertools.product(p, d, q))]

In [13]:
temp = test_df.groupby('Country').agg('size').reset_index()

In [14]:
def getForecast(sales_ts, temp):
    
    best_aic = np.inf
    best_pdq = None
    best_seasonal_pdq = None 
    temp_model = None

    for param in pdq:
        print (param)
        for param_seasonal in seasonal_pdq:
            try:
                temp_model = sm.tsa.statespace.SARIMAX(sales_ts,
                                             order = param,
                                             seasonal_order = param_seasonal,
                                             enforce_stationarity=True,
                                             enforce_invertibility=True)
                results = temp_model.fit()

                if results.aic < best_aic:
                    best_aic = results.aic
                    best_pdq = param
                    best_seasonal_pdq = param_seasonal
            except:
                continue

    best_model = sm.tsa.statespace.SARIMAX(sales_ts,
                                      order=best_pdq,
                                      seasonal_order=best_seasonal_pdq,
                                      enforce_stationarity=True,
                                      enforce_invertibility=True)
    best_results = best_model.fit()
    
    period = int(temp[temp['Country'] == country][0]*5)
    pred_uc_99 = best_results.get_forecast(steps=period, alpha=0.01)
    forecast = pred_uc_99.predicted_mean
    return forecast

In [15]:
forecast_dfs = []
for i in range(len(dataframes)):
    print (df_names[i])
    sales_ts = dataframes[i]['Sales']
    country = dataframes[i]['Country'].iloc[0]
    forecast = getForecast(sales_ts, temp)
    
    period = int(temp[temp['Country'] == country][0]*5)
    time_period = pd.date_range(start = dataframes[i].index[-1], periods=period+1, freq = '7D').values[1:]
    df = pd.DataFrame()
    df['Forecast'] = forecast
    df['Country'] = dataframes[i]['Country'].iloc[0]
    df['Product_ID'] = dataframes[i]['Product_ID'].iloc[0]
    df.index = time_period
    forecast_dfs.append(df)

train_Argentina_1
(0, 0, 0)
(0, 0, 1)
(0, 0, 2)
(0, 1, 0)
(0, 1, 1)
(0, 1, 2)
(0, 2, 0)
(0, 2, 1)
(0, 2, 2)
(1, 0, 0)
(1, 0, 1)
(1, 0, 2)
(1, 1, 0)
(1, 1, 1)
(1, 1, 2)
(1, 2, 0)
(1, 2, 1)
(1, 2, 2)
(2, 0, 0)
(2, 0, 1)
(2, 0, 2)
(2, 1, 0)
(2, 1, 1)
(2, 1, 2)
(2, 2, 0)
(2, 2, 1)
(2, 2, 2)
train_Argentina_2
(0, 0, 0)
(0, 0, 1)
(0, 0, 2)
(0, 1, 0)
(0, 1, 1)
(0, 1, 2)
(0, 2, 0)
(0, 2, 1)
(0, 2, 2)
(1, 0, 0)
(1, 0, 1)
(1, 0, 2)
(1, 1, 0)
(1, 1, 1)
(1, 1, 2)
(1, 2, 0)
(1, 2, 1)
(1, 2, 2)
(2, 0, 0)
(2, 0, 1)
(2, 0, 2)
(2, 1, 0)
(2, 1, 1)
(2, 1, 2)
(2, 2, 0)
(2, 2, 1)
(2, 2, 2)
train_Argentina_3
(0, 0, 0)
(0, 0, 1)
(0, 0, 2)
(0, 1, 0)
(0, 1, 1)
(0, 1, 2)
(0, 2, 0)
(0, 2, 1)
(0, 2, 2)
(1, 0, 0)
(1, 0, 1)
(1, 0, 2)
(1, 1, 0)
(1, 1, 1)
(1, 1, 2)
(1, 2, 0)
(1, 2, 1)
(1, 2, 2)
(2, 0, 0)
(2, 0, 1)
(2, 0, 2)
(2, 1, 0)
(2, 1, 1)
(2, 1, 2)
(2, 2, 0)
(2, 2, 1)
(2, 2, 2)
train_Belgium_2
(0, 0, 0)
(0, 0, 1)
(0, 0, 2)
(0, 1, 0)
(0, 1, 1)
(0, 1, 2)
(0, 2, 0)
(0, 2, 1)
(0, 2, 2)
(1, 0, 0)
(1, 0, 1)
(1, 0, 2)


In [16]:
'''
Formatting every forecast dataframe according to Test Data. Added 'Month', 'Year' column. 
Since the forecast is on week level, I added weekly sales in every month to roll it up to Month level. 
'''
for i in range(len(forecast_dfs)):
    forecast_df = forecast_dfs[i]
    forecast_df['Month'] = forecast_df.index.month
    forecast_df['Year'] = forecast_df.index.year
    temp = forecast_df.groupby('Month').agg('sum')

In [17]:
'''
Joining all monthly forecast dataframes at country, product level into final dataframe('full_df') which can be merged with Test Data
to get final results.
'''
full_df = forecast_dfs[0][['Forecast','Year', 'Month']].groupby(['Year','Month']).agg('sum').reset_index()
full_df['Country'] = forecast_dfs[0].iloc[0]['Country']
full_df['Product_ID'] = forecast_dfs[0].iloc[0]['Product_ID']
for i in range(1,len(forecast_dfs)):
    k = forecast_dfs[i][['Forecast','Year', 'Month']].groupby(['Year','Month']).agg('sum').reset_index()
    k['Country'] = forecast_dfs[i].iloc[i]['Country']
    k['Product_ID'] = forecast_dfs[i].iloc[i]['Product_ID']
    full_df = pd.concat([full_df, k])

In [18]:
# forecast = full_df['Forecast']
# forecast[forecast<0] = 0
# full_df['Forecast'] = forecast

In [19]:
test_data_path =  'yds_test2018.csv'
test_df = pd.read_csv(test_data_path)
test_df.head()

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,NaN
1,79074,2016,5,1,Argentina,NaN
2,79075,2016,6,1,Argentina,NaN
3,79076,2016,7,1,Argentina,NaN
4,79077,2016,8,1,Argentina,NaN


In [20]:
final_df = pd.merge(test_df, full_df, on=['Year', 'Month','Product_ID', 'Country'])

In [21]:
final_df.head()

,S_No,Year,Month,Product_ID,Country,Sales,Forecast
0,79073,2016,4,1,Argentina,NaN,1.059222e+07
1,79074,2016,5,1,Argentina,NaN,7.494083e+06
2,79075,2016,6,1,Argentina,NaN,7.935086e+06
3,79076,2016,7,1,Argentina,NaN,5.593549e+06
4,79077,2016,8,1,Argentina,NaN,5.713509e+06


In [22]:
forecast = final_df['Forecast']
final_df.drop('Forecast', axis=1, inplace=True)
forecast[forecast<0] = min(forecast)
final_df['Sales'] = forecast

In [27]:
final_df.to_csv('submission2.csv', index=False)

# XGBOOST Model

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [6]:
country_dict = {'Argentina':'0', 'Belgium':'1', 'Columbia':'2', 'Denmark':'3', 'Finland':'4', 'England':'5'}
inverse_country_dict = { 0:'Argentina', 1:'Belgium', 2:'Columbia', 3:'Denmark', 4:'Finland', 5:'England'}

In [9]:
train_df = pd.read_csv('data\yds_train2018.csv')

train_df.drop('Week', axis = 1, inplace=True)
train_df.drop('Merchant_ID', axis=1, inplace=True)
train_df.drop('S_No', axis=1, inplace=True)

temp = train_df.groupby(['Country', 'Year', 'Month', 'Product_ID'])
temp = temp.agg('sum')['Sales'].reset_index()
temp.columns = ['Country', 'Year', 'Month', 'Product_ID', 'Monthly_Productwise_sales']
train_df = temp.copy()

expense_df = pd.read_csv('data\promotional_expense.csv')
expense_df.columns = ['Year', 'Month', 'Country', 'Product_ID', 'Expense']
expense_df.head()

train_df = pd.merge(train_df, expense_df, on=['Year', 'Month', 'Country', 'Product_ID'], how='left')
train_df['Country'] = train_df['Country'].apply(lambda x: country_dict[x])
train_df.columns = ['Country', 'Year', 'Month', 'Product_ID', 'Sales', 'Expense']
print (train_df.shape[0])
print ('No of null values in Expense columns: '+ str(len(train_df) - train_df['Expense'].count()))
train_df['Expense'] = train_df['Expense'].fillna(train_df['Expense'].min())

388
No of null values in Expense columns: 52


In [18]:
def concat(l):
    x = ''
    l.sort()
    for y in l:
        x = x+''+str(y)
    return x

def read_date_from_holiday(df):
    year = []
    month = []
    for i in range(df.shape[0]):
        l = df['Date'].iloc[i].split(',')
        year.append(l[0].strip())
        month.append(l[1].strip())
    return year, month

def sMAPE(preds, y):
    score = np.sum( np.abs(preds - y)/ (np.abs(y)+np.abs(preds))*0.5 ) * 100/len(preds)
    return 'sMAPE', score

def get_count_yearwise(df, var):
    temp = df.groupby(['Country','Year', var]).agg('size').reset_index()
    temp.columns = ['Country','Year', var, var+'_country_count']
    df = df.merge(temp, on=['Country','Year', var], how='left')
    return df

def get_count_monthwise(df, var):
    temp = df.groupby(['Country','Month', var]).agg('size').reset_index()
    temp.columns = ['Country','Month', var, var+'_month_count']
    df = df.merge(temp, on=['Country','Month', var], how='left')
    return df

def get_expense_stats(df, var):
    mean = {}
    min = {}
    max = {}
    temp = df.groupby(var)
    print (var)
    for name, group in temp:
        mean[name] = np.mean(group['Expense'])
        min[name] = np.min(group['Expense'])
        max[name] = np.max(group['Expense'])
    return (mean, min, max)

# Preprocessing Holiday Data

In [12]:
holiday_data = pd.read_excel('data\holidays.xlsx')
holiday_data['Country'] = holiday_data['Country'].apply(lambda x: country_dict[x])

year, month = read_date_from_holiday(holiday_data)
holiday_data['Year'] = year
holiday_data['Month'] = month

holiday_data.drop('Date', axis=1, inplace=True)
holiday_data['Year'] = holiday_data['Year'].astype(int)
holiday_data['Month'] = holiday_data['Month'].astype(int)
holiday_data['Holiday'] = holiday_data['Holiday'].astype(str)
holiday_data.head()

from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
holiday_data['Holiday'] = lbl.fit_transform(holiday_data['Holiday'])

holiday_temp = holiday_data.groupby(['Country', 'Year', 'Month', 'Holiday']).agg('size').reset_index()
holiday_temp.columns = ['Country', 'Year', 'Month', 'Holiday', 'Holiday_Period']

unique_holidays_monthwise = holiday_temp.groupby(['Country', 'Year', 'Month']).agg('size').reset_index()


check = holiday_temp.groupby(['Country', 'Year', 'Month'])

concatinated_holidays = []
for name, group in check:
    concatinated_holidays.append(concat(group['Holiday'].values))

unique_holidays_monthwise['Holiday'] = concatinated_holidays

unique_holidays_monthwise.columns = ['Country', 'Year', 'Month', 'No_of_unique_festivals', 'Holiday']
holiday_monthwise = holiday_data.groupby(['Country', 'Year', 'Month']).agg('size').reset_index()
holiday_monthwise.columns = ['Country', 'Year', 'Month', 'Holiday_count_every_month']

modified_holiday_data = pd.merge(unique_holidays_monthwise, holiday_monthwise, on=['Country', 'Year', 'Month'])
train_df = pd.merge(train_df, modified_holiday_data, on=['Year', 'Month', 'Country'], how='left')
train_df = train_df.drop_duplicates().reset_index()
train_df.drop('index', axis=1, inplace=True)

train_df['No_of_unique_festivals'] = train_df['No_of_unique_festivals'].fillna(0)
train_df['Holiday'] = train_df['Holiday'].fillna('999')
train_df['Holiday_count_every_month'] = train_df['Holiday_count_every_month'].fillna(0)

In [16]:
train_df.head()

,Country,Year,Month,Product_ID,Sales,Expense,No_of_unique_festivals,Holiday
0,0,2013,1,1,34346025.00,14749.307,1.0,0
1,0,2013,1,2,2751851.48,1329.374,1.0,0
2,0,2013,2,1,32005575.00,12187.566,1.0,2
3,0,2013,2,2,2804313.12,1315.006,1.0,2
4,0,2013,3,1,32530050.00,13076.579,3.0,53


# Pre-process Test Data

In [13]:
test_df = pd.read_csv('data\yds_test2018.csv')
test_df.to_csv('data\submission0.csv')

test_df.drop('S_No', axis=1, inplace=True)
test_df.drop('Sales', axis=1, inplace=True)
test_df = pd.merge(test_df, expense_df, on=['Year', 'Month', 'Country', 'Product_ID'], how='left')
test_df['Country'] = test_df['Country'].apply(lambda x: country_dict[x])

test_df = pd.merge(test_df, modified_holiday_data, on=['Year', 'Month', 'Country'], how='left')
test_df = test_df.drop_duplicates().reset_index()
test_df.drop('index', axis=1, inplace=True)

test_df['No_of_unique_festivals'] = test_df['No_of_unique_festivals'].fillna(0)
test_df['Holiday'] = test_df['Holiday'].fillna('999')
test_df['Holiday_count_every_month'] = test_df['Holiday_count_every_month'].fillna(0)

test_df = test_df[['Country','Year', 'Month', 'Product_ID', 'Expense', 'No_of_unique_festivals', 'Holiday', 'Holiday_count_every_month']]

train_df['Country'] = train_df['Country'].astype(int)
train_df['Holiday'] = train_df['Holiday'].astype(int)

test_df['Country'] = test_df['Country'].astype(int)
test_df['Holiday'] = test_df['Holiday'].astype(int)

test_df['Expense'] = test_df['Expense'].fillna(0)

In [14]:
train_df['Holiday'] = lbl.fit_transform(train_df['Holiday'])
test_df['Holiday'] = lbl.fit_transform(test_df['Holiday'])

In [15]:
test_df.drop('Holiday_count_every_month', axis=1, inplace=True)
train_df.drop('Holiday_count_every_month', axis=1, inplace=True)

In [17]:
test_df.head()

,Country,Year,Month,Product_ID,Expense,No_of_unique_festivals,Holiday
0,0,2016,4,1,8214.875,1.0,11
1,0,2016,5,1,10777.878,2.0,26
2,0,2016,6,1,10320.673,2.0,24
3,0,2016,7,1,7377.587,1.0,8
4,0,2016,8,1,9805.705,1.0,7


In [19]:
count_categories = ['Product_ID']
for x in count_categories:
    train_df = get_count_yearwise(train_df, x)
    train_df = get_count_monthwise(train_df, x)
    test_df = get_count_monthwise(test_df, x)
    test_df = get_count_yearwise(test_df, x)

In [20]:
categories = ['Month', 'Product_ID', 'Country']

for x in categories:
    mean, min_, max_ = get_expense_stats(train_df, x)
    train_df[x+'_mean'], train_df[x+'_min'] , train_df[x+'_max']  = train_df[x].apply(lambda x: mean[x]), train_df[x].apply(lambda x: min_[x]), train_df[x].apply(lambda x: max_[x])

for x in categories:
    mean, min_, max_ = get_expense_stats(test_df, x)    
    test_df[x+'_mean'], test_df[x+'_min'] , test_df[x+'_max']  = test_df[x].apply(lambda x: mean[x]), test_df[x].apply(lambda x: min_[x]), test_df[x].apply(lambda x: max_[x])

Month
Product_ID
Country
Month
Product_ID
Country


In [24]:
#Output = train_df['Sales']
#train_df.drop('Sales', axis=1, inplace=True)
train_X, test_X, train_y, test_y = train_test_split(train_df, Output, test_size=0.2, random_state=123)

In [28]:
train_X.shape

(310, 18)

In [128]:
model = xgb.XGBRegressor()
model.fit(train_X, train_y)
val_pred = model.predict(test_X)

sMAPE(val_pred, test_y.values)

('sMAPE', 11.067373237614683)

In [190]:
train_X, test_X, train_y, test_y = train_test_split(train_df, Output, test_size=0.3, random_state=123)
model_1 = xgb.XGBRegressor(n_estimators= 50,
                           learning_rate = 0.1,
                           max_depth = 6,
                           colsample_bylevel = 0.6,
                           feval = sMAPE,
                           maximize=False
                          )
model_1.fit(train_X.values, train_y.values)
test_pred_1 = model_1.predict(test_X.values)
sMAPE(test_pred_1, test_y.values)

('sMAPE', 4.138804128817378)

In [191]:
train_X, test_X, train_y, test_y = train_test_split(train_df, Output, test_size=0.3, random_state=123)
model_2 = xgb.XGBRegressor(n_estimators= 50,
                           learning_rate = 0.1,
                           max_depth = 6,
                           colsample_bylevel = 0.5,
                           feval = sMAPE,
                           maximize=False
                          )
model_2.fit(train_X.values, train_y.values, eval_set=[(test_X.values, test_y.values)], verbose=0)
test_pred_2 = model_2.predict(test_X.values)
sMAPE(test_pred_2, test_y.values)

('sMAPE', 3.9222568782812703)

In [192]:
final_df = pd.read_csv('submissions/submission0.csv')
final_df.drop('Unnamed: 0', axis=1, inplace=True)
final_df.drop('Sales', axis=1, inplace=True)

pred6 = model_1.predict(test_df.values)*0.3 + model_2.predict(test_df.values) * 0.7
pred6 = np.abs(pred6)

final_df['Sales'] = pred6

final_df.to_csv('submissions/submission3.csv', index=False)

# Averaging  ARIMA, 2XGBoost Models

In [195]:
df1 = pd.read_csv('submissions/submission2.csv')
df2 = pd.read_csv('submissions/submission3.csv')

final_pred = df1['Sales']*0.5 + df2['Sales']*0.5

final_df = pd.read_csv('submissions/submission0.csv')
final_df.drop('Unnamed: 0', axis=1, inplace=True)
final_df.drop('Sales', axis=1, inplace=True)
final_df['Sales'] =  final_pred

final_df.to_csv('submissions/shariq_final_submission.csv', index=False)